In [111]:
import numpy as np
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Lambda
from keras.constraints import maxnorm
from keras.optimizers import adam
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

In [2]:
np.random.seed(10)

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [4]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [5]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [116]:
import tensorflow as tf
from sklearn.utils.random import sample_without_replacement
from sklearn.utils import check_random_state

import scipy.sparse as sp

def gaussian_random_tensor(n_components, n_features, random_state=None):
    rng = check_random_state(random_state)
    components = rng.normal(loc=0.0,
                            scale=1.0 / np.sqrt(n_components),
                            size=(n_components, n_features))
    components = components.T
    # components = np.sign(components)
    return (tf.convert_to_tensor(components, dtype = 'float32'))

def _check_density(density, n_features):
    """Factorize density check according to Li et al."""
    if density == 'auto':
        density = 1 / np.sqrt(n_features)
    elif density <= 0 or density > 1:
        raise ValueError("Expected density in range ]0, 1], got: %r"
                         % density)
    return density

def sparse_random_tensor(n_components, n_features, density='auto',
                         random_state=None):
 
    density = _check_density(density, n_features)
    rng = check_random_state(random_state)
    if density == 1:
        # skip index generation if totally dense
        components = rng.binomial(1, 0.5, (n_components, n_features)) * 2 - 1
        return 1 / np.sqrt(n_components) * components
    else:
        indices = []
        offset = 0
        indptr = [offset]
        for i in range(n_components):
            # find the indices of the non-zero components for row i
            n_nonzero_i = rng.binomial(n_features, density)
            indices_i = sample_without_replacement(n_features, n_nonzero_i,
                                                   random_state=rng)
            indices.append(indices_i)
            offset += n_nonzero_i
            indptr.append(offset)
        indices = np.concatenate(indices)
        # Among non zero components the probability of the sign is 50%/50%
        data = rng.binomial(1, 0.5, size=np.size(indices)) * 2 - 1
        # build the CSR structure by concatenating the rows
        components = sp.csr_matrix((data, indices, indptr),
                                   shape=(n_components, n_features))
        return tf.convert_to_tensor(np.sqrt(1 / density) / np.sqrt(n_components) * components)


def project(x, ncomp): #ncomp is the number of dimensions we want to shrink to 
    features = K.int_shape(x)[1]
    Y = gaussian_random_tensor(ncomp, features)
    X_new = K.dot(x, Y)
    return X_new

In [117]:
dims = [2048, 1536, 1024, 512, 256, 128, 64]
import time
accuracy = []
size = []
times = []
epochs = 10

for d in dims:
    start = time.time()
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    if d != 2048:
        model.add(Lambda (lambda x: project(x, d)))
    model.add(Dense(d, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    h = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)
    a = model.evaluate(X_test, y_test, verbose=0)[1]
    
    end = time.time()
    t = end - start
    accuracy.append(a)
    size.append(d)
    times.append(t)

import pandas as pd
df = pd.DataFrame({'Size': size, 'Accuracy': accuracy, 'Runtime': times})
    

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 426s - loss: 1.4002 - acc: 0.4936 - val_loss: 1.1321 - val_acc: 0.5936
Epoch 2/10
50000/50000 [==============================] - 418s - loss: 1.0184 - acc: 0.6384 - val_loss: 0.9625 - val_acc: 0.6591
Epoch 3/10
50000/50000 [==============================] - 425s - loss: 0.8324 - acc: 0.7054 - val_loss: 0.8803 - val_acc: 0.6872
Epoch 4/10
50000/50000 [==============================] - 418s - loss: 0.6960 - acc: 0.7534 - val_loss: 0.8583 - val_acc: 0.7025
Epoch 5/10
50000/50000 [==============================] - 418s - loss: 0.5707 - acc: 0.8000 - val_loss: 0.8866 - val_acc: 0.7083
Epoch 6/10
50000/50000 [==============================] - 414s - loss: 0.4624 - acc: 0.8378 - val_loss: 0.9811 - val_acc: 0.6947
Epoch 7/10
50000/50000 [==============================] - 416s - loss: 0.3813 - acc: 0.8666 - val_loss: 0.9545 - val_acc: 0.7085
Epoch 8/10
50000/50000 [=======================

50000/50000 [==============================] - 282s - loss: 1.6757 - acc: 0.3885 - val_loss: 1.4546 - val_acc: 0.4762
Epoch 2/10
50000/50000 [==============================] - 277s - loss: 1.4107 - acc: 0.4892 - val_loss: 1.3264 - val_acc: 0.5183
Epoch 3/10
50000/50000 [==============================] - 277s - loss: 1.3222 - acc: 0.5260 - val_loss: 1.2925 - val_acc: 0.5451
Epoch 4/10
50000/50000 [==============================] - 278s - loss: 1.2704 - acc: 0.5437 - val_loss: 1.2283 - val_acc: 0.5618
Epoch 5/10
50000/50000 [==============================] - 278s - loss: 1.2325 - acc: 0.5601 - val_loss: 1.2139 - val_acc: 0.5694
Epoch 6/10
50000/50000 [==============================] - 277s - loss: 1.2008 - acc: 0.5711 - val_loss: 1.1903 - val_acc: 0.5726
Epoch 7/10
50000/50000 [==============================] - 277s - loss: 1.1786 - acc: 0.5776 - val_loss: 1.1391 - val_acc: 0.5950
Epoch 8/10
50000/50000 [==============================] - 277s - loss: 1.1590 - acc: 0.5862 - val_loss: 1.17

In [123]:
df['time_saved'] = (max(df['Runtime']) - df['Runtime']) / max(df['Runtime'])
df['acc_lost'] = (max(df['Accuracy']) - df['Accuracy'])
df.head()
df.to_csv('Cifar1.csv')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

sns.set_style("whitegrid")
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')

plt.subplot(1, 2, 2)
plt.plot(h.history['acc'])
plt.plot(h.history['val_acc'])
plt.legend(['acc', 'val_acc'])
plt.ylabel('acc')
plt.xlabel('epoch')